In [1]:
# transformer-decoder with no Seq2Seq component (autoregressive)

# no value embedding
# sine-cosine positional encoding on the hour, day, and month of timestamp
# modified transformer-encoder layer for masked self-attention

# conduct an architecture test similar to the one on the transformer-encoder in
# deep_transformer_model_for_tsf_XX.ipynb

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf

In [4]:
print(tf.__version__)

2.4.1


In [5]:
# required for TFA MultiHeadAttention
import typing
import warnings

In [6]:
# MHA class from TensorFlow AddOns source
# it is compatible with TF 1.15 for CloudTPU usage

In [7]:
class MultiHeadAttention(tf.keras.layers.Layer):
    r"""MultiHead Attention layer.
    Defines the MultiHead Attention operation as described in
    [Attention Is All You Need](https://arxiv.org/abs/1706.03762) which takes
    in the tensors `query`, `key`, and `value`, and returns the dot-product attention
    between them:
    >>> mha = MultiHeadAttention(head_size=128, num_heads=12)
    >>> query = np.random.rand(3, 5, 4) # (batch_size, query_elements, query_depth)
    >>> key = np.random.rand(3, 6, 5) # (batch_size, key_elements, key_depth)
    >>> value = np.random.rand(3, 6, 6) # (batch_size, key_elements, value_depth)
    >>> attention = mha([query, key, value]) # (batch_size, query_elements, value_depth)
    >>> attention.shape
    TensorShape([3, 5, 6])
    If `value` is not given then internally `value = key` will be used:
    >>> mha = MultiHeadAttention(head_size=128, num_heads=12)
    >>> query = np.random.rand(3, 5, 5) # (batch_size, query_elements, query_depth)
    >>> key = np.random.rand(3, 6, 10) # (batch_size, key_elements, key_depth)
    >>> attention = mha([query, key]) # (batch_size, query_elements, key_depth)
    >>> attention.shape
    TensorShape([3, 5, 10])
    Args:
        head_size: int, dimensionality of the `query`, `key` and `value` tensors
            after the linear transformation.
        num_heads: int, number of attention heads.
        output_size: int, dimensionality of the output space, if `None` then the
            input dimension of `value` or `key` will be used,
            default `None`.
        dropout: float, `rate` parameter for the dropout layer that is
            applied to attention after softmax,
        default `0`.
        use_projection_bias: bool, whether to use a bias term after the linear
            output projection.
        return_attn_coef: bool, if `True`, return the attention coefficients as
            an additional output argument.
        kernel_initializer: initializer, initializer for the kernel weights.
        kernel_regularizer: regularizer, regularizer for the kernel weights.
        kernel_constraint: constraint, constraint for the kernel weights.
        bias_initializer: initializer, initializer for the bias weights.
        bias_regularizer: regularizer, regularizer for the bias weights.
        bias_constraint: constraint, constraint for the bias weights.
    Call Args:
        inputs:  List of `[query, key, value]` where
            * `query`: Tensor of shape `(..., query_elements, query_depth)`
            * `key`: `Tensor of shape '(..., key_elements, key_depth)`
            * `value`: Tensor of shape `(..., key_elements, value_depth)`, optional, if not given `key` will be used.
        mask: a binary Tensor of shape `[batch_size?, num_heads?, query_elements, key_elements]`
        which specifies which query elements can attend to which key elements,
        `1` indicates attention and `0` indicates no attention.
    Output shape:
        * `(..., query_elements, output_size)` if `output_size` is given, else
        * `(..., query_elements, value_depth)` if `value` is given, else
        * `(..., query_elements, key_depth)`
    """

    def __init__(
        self,
        head_size: int,
        num_heads: int,
        output_size: int = None,
        dropout: float = 0.0,
        use_projection_bias: bool = True,
        return_attn_coef: bool = False,
        kernel_initializer: typing.Union[str, typing.Callable] = "glorot_uniform",
        kernel_regularizer: typing.Union[str, typing.Callable] = None,
        kernel_constraint: typing.Union[str, typing.Callable] = None,
        bias_initializer: typing.Union[str, typing.Callable] = "zeros",
        bias_regularizer: typing.Union[str, typing.Callable] = None,
        bias_constraint: typing.Union[str, typing.Callable] = None,
        **kwargs,
    ):
        warnings.warn(
            "`MultiHeadAttention` will be deprecated in Addons 0.13. "
            "Please use `tf.keras.layers.MultiHeadAttention` instead.",
            DeprecationWarning,
        )

        super().__init__(**kwargs)

        if output_size is not None and output_size < 1:
            raise ValueError("output_size must be a positive number")

        self.head_size = head_size
        self.num_heads = num_heads
        self.output_size = output_size
        self.use_projection_bias = use_projection_bias
        self.return_attn_coef = return_attn_coef

        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)

        self.dropout = tf.keras.layers.Dropout(dropout)
        self._dropout_rate = dropout

    def build(self, input_shape):

        num_query_features = input_shape[0][-1]
        num_key_features = input_shape[1][-1]
        num_value_features = (
            input_shape[2][-1] if len(input_shape) > 2 else num_key_features
        )
        output_size = (
            self.output_size if self.output_size is not None else num_value_features
        )

        self.query_kernel = self.add_weight(
            name="query_kernel",
            shape=[self.num_heads, num_query_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.key_kernel = self.add_weight(
            name="key_kernel",
            shape=[self.num_heads, num_key_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.value_kernel = self.add_weight(
            name="value_kernel",
            shape=[self.num_heads, num_value_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.projection_kernel = self.add_weight(
            name="projection_kernel",
            shape=[self.num_heads, self.head_size, output_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )

        if self.use_projection_bias:
            self.projection_bias = self.add_weight(
                name="projection_bias",
                shape=[output_size],
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )
        else:
            self.projection_bias = None

        super().build(input_shape)

    def call(self, inputs, training=None, mask=None):

        # einsum nomenclature
        # ------------------------
        # N = query elements
        # M = key/value elements
        # H = heads
        # I = input features
        # O = output features

        query = inputs[0]
        key = inputs[1]
        value = inputs[2] if len(inputs) > 2 else key

        # verify shapes
        if key.shape[-2] != value.shape[-2]:
            raise ValueError(
                "the number of elements in 'key' must be equal to the same as the number of elements in 'value'"
            )

        if mask is not None:
            if len(mask.shape) < 2:
                raise ValueError("'mask' must have atleast 2 dimensions")
            if query.shape[-2] != mask.shape[-2]:
                raise ValueError(
                    "mask's second to last dimension must be equal to the number of elements in 'query'"
                )
            if key.shape[-2] != mask.shape[-1]:
                raise ValueError(
                    "mask's last dimension must be equal to the number of elements in 'key'"
                )

        # Linear transformations
        query = tf.einsum("...NI , HIO -> ...NHO", query, self.query_kernel)
        key = tf.einsum("...MI , HIO -> ...MHO", key, self.key_kernel)
        value = tf.einsum("...MI , HIO -> ...MHO", value, self.value_kernel)

        # Scale dot-product, doing the division to either query or key
        # instead of their product saves some computation
        depth = tf.constant(self.head_size, dtype=query.dtype)
        query /= tf.sqrt(depth)

        # Calculate dot product attention
        logits = tf.einsum("...NHO,...MHO->...HNM", query, key)

        # apply mask
        if mask is not None:
            mask = tf.cast(mask, tf.float32)

            # possibly expand on the head dimension so broadcasting works
            if len(mask.shape) != len(logits.shape):
                mask = tf.expand_dims(mask, -3)

            logits += -10e9 * (1.0 - mask)

        attn_coef = tf.nn.softmax(logits)

        # attention dropout
        attn_coef_dropout = self.dropout(attn_coef, training=training)

        # attention * value
        multihead_output = tf.einsum("...HNM,...MHI->...NHI", attn_coef_dropout, value)

        # Run the outputs through another linear projection layer. Recombining heads
        # is automatically done.
        output = tf.einsum(
            "...NHI,HIO->...NO", multihead_output, self.projection_kernel
        )

        if self.projection_bias is not None:
            output += self.projection_bias

        if self.return_attn_coef:
            return output, attn_coef
        else:
            return output

    def compute_output_shape(self, input_shape):
        num_value_features = (
            input_shape[2][-1] if len(input_shape) > 2 else input_shape[1][-1]
        )
        output_size = (
            self.output_size if self.output_size is not None else num_value_features
        )

        output_shape = input_shape[0][:-1] + (output_size,)

        if self.return_attn_coef:
            num_query_elements = input_shape[0][-2]
            num_key_elements = input_shape[1][-2]
            attn_coef_shape = input_shape[0][:-2] + (
                self.num_heads,
                num_query_elements,
                num_key_elements,
            )

            return output_shape, attn_coef_shape
        else:
            return output_shape

    def get_config(self):
        config = super().get_config()

        config.update(
            head_size=self.head_size,
            num_heads=self.num_heads,
            output_size=self.output_size,
            dropout=self._dropout_rate,
            use_projection_bias=self.use_projection_bias,
            return_attn_coef=self.return_attn_coef,
            kernel_initializer=tf.keras.initializers.serialize(self.kernel_initializer),
            kernel_regularizer=tf.keras.regularizers.serialize(self.kernel_regularizer),
            kernel_constraint=tf.keras.constraints.serialize(self.kernel_constraint),
            bias_initializer=tf.keras.initializers.serialize(self.bias_initializer),
            bias_regularizer=tf.keras.regularizers.serialize(self.bias_regularizer),
            bias_constraint=tf.keras.constraints.serialize(self.bias_constraint),
        )

        return config

In [8]:
# the autoregressive version of the transformer-decoder does not use the Seq2Seq intermediate layer
# as there is no transformer-encoder component sending hidden states, therefore
# having only a self-attention layer and a position-wise feed-forward layer,
# the autoregressive transformer-decoder is, in fact, a transformer-encoder

# the only important modification is the masked self-attention layer

# masked self-attention layer seems to be already implemented in
# MHA module from TensorFlow AddOns, then will be added to the EncoderLayer class

In [9]:
# base transformer encoder layer from # https://keras.io/examples/nlp/text_classification_with_transformer/
# modified to include masked self attention

# ToDo: get the number of timesteps from the input shape
# in the meantime, pass this value as an argument for the encoder layer
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, n_timesteps, embed_dim, num_heads, ff_dim, dropout=0.1):
        super(EncoderLayer, self).__init__()
        # multi-head attention initialization
        self.attention_layer = MultiHeadAttention(head_size=embed_dim, num_heads=num_heads)
        self.ff_layer = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"),
             tf.keras.layers.Dense(embed_dim)]
        )
        self.add_norm_layer_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.add_norm_layer_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout_1 = tf.keras.layers.Dropout(dropout)
        self.dropout_2 = tf.keras.layers.Dropout(dropout)
        # mask for self-attention
        self.mask = tf.convert_to_tensor(np.tril(np.ones([n_timesteps, n_timesteps]), 0), dtype=tf.float32)

        
    def call(self, inputs, training):
        # mask for self-attention is passed to MHA on call
        attention_output = self.attention_layer([inputs, inputs], mask=self.mask)
        attention_output = self.dropout_1(attention_output, training=training)
        input_to_ffn = self.add_norm_layer_1(inputs + attention_output)
        ffn_output = self.ff_layer(input_to_ffn)
        ffn_output = self.dropout_2(ffn_output, training=training)
        return self.add_norm_layer_2(input_to_ffn + ffn_output)

In [10]:
ts = pd.read_pickle("/home/developer/gcp/cbidmltsf/timeseries/CPE04115_H_kw_20201021084001/ts.pkl")

In [11]:
ts.head()

,kw_scaled
timestamp,
2016-01-01 00:00:00,0.274317
2016-01-01 01:00:00,0.217363
2016-01-01 02:00:00,0.168545
2016-01-01 03:00:00,0.122996
2016-01-01 04:00:00,0.080440


In [12]:
# length of time series
total_lectures = ts['kw_scaled'].count()
total_lectures

22629

In [13]:
# build a training dataset
# features: m consecutive lectures with their timestamps
# target: m consecutive lectures (lectures in features, shifted by 1 to the future)

In [14]:
# FEATURE ENGINEERING

In [15]:
# define the embedding dimension, the lenght of tranining, evaluation, and test datasets
# use the most of the 22K+ lectures in the original time series

In [16]:
# length of input sequence, hours in a week
m = 168

In [17]:
# prepare sine-cosine positional encoding for the input sequence
hours_in_day = 24
days_in_month = 30
months_in_year = 12

In [18]:
total_lectures*np.array([0.7, 0.2, 0.1])

array([15840.3,  4525.8,  2262.9])

In [19]:
# from the previous dataset split, use the following indexes for building datasets

# 15000 rows in training dataset
train_start = 0
train_end = 15000

# 4000 rows in evaluation dataset
eval_start = 16000
eval_end = 20000

# 1000 rows in test dataset
test_start = 21000
test_end = 22000

In [26]:
# get a collection of examples for training
# use a range on the time series index

train_range = np.arange(train_start, train_end)

features_list = list()
targets_list = list()

for start in train_range:
    end = start + m
    values = np.expand_dims(ts[start:end]['kw_scaled'].values, axis=1)
    target_values = np.expand_dims(ts[1+start:1+end]['kw_scaled'].values, axis=1)
    
    timestamps_hour = ts[start:end].index.hour
    timestamps_day = ts[start:end].index.day
    timestamps_month = ts[start:end].index.month
    
    sin_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    cos_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    sin_day = np.expand_dims(np.sin(2*np.pi*timestamps_day/days_in_month), axis=1)
    cos_day = np.expand_dims(np.cos(2*np.pi*timestamps_day/days_in_month), axis=1)
    sin_month = np.expand_dims(np.sin(2*np.pi*timestamps_month/months_in_year), axis=1)
    cos_month = np.expand_dims(np.cos(2*np.pi*timestamps_month/months_in_year), axis=1)
    
    feature_row = np.concatenate((values, sin_hour, cos_hour, sin_day, cos_day, sin_month, cos_month), axis=1)    

    features_list.append(feature_row)
    targets_list.append(target_values)

x_train = np.array(features_list)
y_train = np.array(targets_list)

In [69]:
# save the training NumPy arrays
with open('data/x_train.npy', 'wb') as filename:
    np.save(filename, x_train)

with open('data/y_train.npy', 'wb') as filename:
    np.save(filename, y_train)

In [20]:
# load the training NumPy arrays
with open('data/x_train.npy', 'rb') as filename:
    x_train = np.load(filename)

with open('data/y_train.npy', 'rb') as filename:
    y_train = np.load(filename)

In [21]:
x_train.shape, y_train.shape

((15000, 168, 7), (15000, 168, 1))

In [30]:
# get a collection of examples for evaluation
# use a range on the time series index
eval_range = np.arange(eval_start, eval_end)

features_list = list()
targets_list = list()

for start in eval_range:
    end = start + m
    values = np.expand_dims(ts[start:end]['kw_scaled'].values, axis=1)
    target_values = np.expand_dims(ts[1+start:1+end]['kw_scaled'].values, axis=1)
    
    timestamps_hour = ts[start:end].index.hour
    timestamps_day = ts[start:end].index.day
    timestamps_month = ts[start:end].index.month
    
    sin_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    cos_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    sin_day = np.expand_dims(np.sin(2*np.pi*timestamps_day/days_in_month), axis=1)
    cos_day = np.expand_dims(np.cos(2*np.pi*timestamps_day/days_in_month), axis=1)
    sin_month = np.expand_dims(np.sin(2*np.pi*timestamps_month/months_in_year), axis=1)
    cos_month = np.expand_dims(np.cos(2*np.pi*timestamps_month/months_in_year), axis=1)
    
    feature_row = np.concatenate((values, sin_hour, cos_hour, sin_day, cos_day, sin_month, cos_month), axis=1)    

    features_list.append(feature_row)
    targets_list.append(target_values)

x_eval = np.array(features_list)
y_eval = np.array(targets_list)

In [67]:
# save the evaluation NumPy arrays
with open('data/x_eval.npy', 'wb') as filename:
    np.save(filename, x_eval)

with open('data/y_eval.npy', 'wb') as filename:
    np.save(filename, y_eval)

In [22]:
# load the evaluation NumPy arrays
with open('data/x_eval.npy', 'rb') as filename:
    x_eval = np.load(filename)

with open('data/y_eval.npy', 'rb') as filename:
    y_eval = np.load(filename)

In [23]:
x_eval.shape, y_eval.shape

((4000, 168, 7), (4000, 168, 1))

In [62]:
# get a collection of examples for test
# use a range on the time series index
test_range = np.arange(test_start, test_end)

features_list = list()
targets_list = list()

for start in test_range:
    end = start + m
    values = np.expand_dims(ts[start:end]['kw_scaled'].values, axis=1)
    target_values = np.expand_dims(ts[1+start:1+end]['kw_scaled'].values, axis=1)
    
    timestamps_hour = ts[start:end].index.hour
    timestamps_day = ts[start:end].index.day
    timestamps_month = ts[start:end].index.month
    
    sin_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    cos_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    sin_day = np.expand_dims(np.sin(2*np.pi*timestamps_day/days_in_month), axis=1)
    cos_day = np.expand_dims(np.cos(2*np.pi*timestamps_day/days_in_month), axis=1)
    sin_month = np.expand_dims(np.sin(2*np.pi*timestamps_month/months_in_year), axis=1)
    cos_month = np.expand_dims(np.cos(2*np.pi*timestamps_month/months_in_year), axis=1)
    
    feature_row = np.concatenate((values, sin_hour, cos_hour, sin_day, cos_day, sin_month, cos_month), axis=1)    

    features_list.append(feature_row)
    targets_list.append(target_values)

x_test = np.array(features_list)
y_test = np.array(targets_list)

In [65]:
# save the test NumPy arrays
with open('data/x_test.npy', 'wb') as filename:
    np.save(filename, x_test)
    
with open('data/y_test.npy', 'wb') as filename:
    np.save(filename, y_test)

In [24]:
# load the test NumPy arrays
with open('data/x_test.npy', 'rb') as filename:
    x_test = np.load(filename)

with open('data/y_test.npy', 'rb') as filename:
    y_test = np.load(filename)

In [25]:
x_test.shape, y_test.shape

((1000, 168, 7), (1000, 168, 1))

In [26]:
# architecture details according to the Klingenbrunn experiment
# (including notes to further modifications on the basic autoregressive model)

In [27]:
# number of timesteps is the length of the input sequence,
# is the embedding dimension from SLDB
num_timesteps = m

In [28]:
# number of features is the active load value (main feature)
# plus the six components of the sine-cosine positional encoding on hour, day, month

# important: there is no value embedding, therefore d_model is very low
d_model = 7

# ToDo: use value embedding to a high-dimensional space and compare results
# ToDo: use a different positional encoding system and compare results

In [29]:
# as long as there is no value embedding, neither convolutional nor dense layers are required

In [30]:
# input layer for Keras functional
input_layer = tf.keras.layers.Input(shape=(num_timesteps, d_model))
input_layer

<KerasTensor: shape=(None, 168, 7) dtype=float32 (created by layer 'input_1')>

In [31]:
input_to_transformer_block = input_layer
input_to_transformer_block

<KerasTensor: shape=(None, 168, 7) dtype=float32 (created by layer 'input_1')>

In [32]:
num_heads = 2
ff_dim = 1024
dropout = 0.2

In [33]:
encoder_layer_1 = EncoderLayer(n_timesteps=num_timesteps,
                               embed_dim=d_model,
                               num_heads=num_heads,
                               ff_dim=ff_dim,
                               dropout=dropout)

/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: `MultiHeadAttention` will be deprecated in Addons 0.13. Please use `tf.keras.layers.MultiHeadAttention` instead.


In [34]:
encoder_layer_2 = EncoderLayer(n_timesteps=num_timesteps,
                               embed_dim=d_model,
                               num_heads=num_heads,
                               ff_dim=ff_dim,
                               dropout=dropout)

/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: `MultiHeadAttention` will be deprecated in Addons 0.13. Please use `tf.keras.layers.MultiHeadAttention` instead.


In [35]:
output_from_encoder_1 = encoder_layer_1(input_to_transformer_block)
output_from_encoder_1

<KerasTensor: shape=(None, 168, 7) dtype=float32 (created by layer 'encoder_layer')>

In [36]:
output_from_encoder_2 = encoder_layer_2(output_from_encoder_1)
output_from_encoder_2

<KerasTensor: shape=(None, 168, 7) dtype=float32 (created by layer 'encoder_layer_1')>

In [37]:
# Klingenbrunn uses a linear layer to decode the output_from_encoder
# from (?, num_timesteps, num_features) to (?, num_timesteps, 1)

# the equivalent operation in TensorFlow is a TimeDistributed Dense layer to 1

In [38]:
units_in_first_dense = 1
first_dense = tf.keras.layers.Dense(units_in_first_dense, activation="sigmoid")

In [39]:
distributed_first_dense = tf.keras.layers.TimeDistributed(first_dense)(output_from_encoder_2)
distributed_first_dense

<KerasTensor: shape=(None, 168, 1) dtype=float32 (created by layer 'time_distributed')>

In [40]:
model = tf.keras.Model(inputs=input_layer, outputs=distributed_first_dense)

In [41]:
model.compile("adam", "mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [42]:
history = model.fit(
    x_train, y_train, batch_size=32, epochs=10, validation_data=(x_eval, y_eval)
)

Epoch 1/10
469/469 [==============================] - 65s 134ms/step - loss: 0.0213 - root_mean_squared_error: 0.1425 - val_loss: 0.0028 - val_root_mean_squared_error: 0.0528
Epoch 2/10
469/469 [==============================] - 62s 133ms/step - loss: 0.0055 - root_mean_squared_error: 0.0742 - val_loss: 0.0024 - val_root_mean_squared_error: 0.0487
Epoch 3/10
469/469 [==============================] - 62s 132ms/step - loss: 0.0040 - root_mean_squared_error: 0.0636 - val_loss: 0.0024 - val_root_mean_squared_error: 0.0495
Epoch 4/10
469/469 [==============================] - 62s 133ms/step - loss: 0.0035 - root_mean_squared_error: 0.0591 - val_loss: 0.0022 - val_root_mean_squared_error: 0.0470
Epoch 5/10
469/469 [==============================] - 62s 133ms/step - loss: 0.0032 - root_mean_squared_error: 0.0564 - val_loss: 0.0023 - val_root_mean_squared_error: 0.0478
Epoch 6/10
469/469 [==============================] - 62s 133ms/step - loss: 0.0029 - root_mean_squared_error: 0.0540 - val_l

In [43]:
# inference process on unseen data, that means on the test dataset

In [44]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [45]:
# use naming conventions from Klingenbrunn

In [46]:
# the true active load values, plus the six positional encodings
# in the first row of the test dataset features (original ts indexes are 21000-21167)
src = tf.expand_dims(x_test[0, :, :], axis=0)
src.shape

TensorShape([1, 168, 7])

In [47]:
# the true active load values, plus the six positional encodings
# in the forecast-window-sized timesteps following the source (original ts indexes are 21168-21191)

In [48]:
target = tf.expand_dims(x_test[168, :forecast_window, :], axis=0)
target.shape

TensorShape([1, 24, 7])

In [49]:
# verify that active load values in the source and the target are contiguous in the time series
# that means, target is composed with the first timesteps of features
# in the rows 168-191 of test dataset

In [50]:
x_test[168:192, 0, 0]

array([0.26612255, 0.23641237, 0.24432548, 0.32326286, 0.36856605,
       0.45449848, 0.56519907, 0.68640037, 0.74005594, 0.84452149,
       0.86219931, 0.86331182, 0.87358708, 0.94115296, 0.89783385,
       0.90333212, 0.8544241 , 0.89447384, 0.90186633, 0.83024086,
       0.67798756, 0.54712923, 0.42006322, 0.33738621])

In [51]:
target[0, :, 0]

<tf.Tensor: shape=(24,), dtype=float64, numpy=
array([0.26612255, 0.23641237, 0.24432548, 0.32326286, 0.36856605,
       0.45449848, 0.56519907, 0.68640037, 0.74005594, 0.84452149,
       0.86219931, 0.86331182, 0.87358708, 0.94115296, 0.89783385,
       0.90333212, 0.8544241 , 0.89447384, 0.90186633, 0.83024086,
       0.67798756, 0.54712923, 0.42006322, 0.33738621])>

In [52]:
# therefore, source and target tensors are ready for inference process

In [71]:
next_input_model = src

In [72]:
# do not use the all_predictions array from Klingenbrunn, just the predictions on the forecast window
predictions_list = list()

In [73]:
# once tested, put the following lines on an iterative cycle
# for i in range(forecast_window -1): ??

In [74]:
# first pass
i = 0

# src is the feature set of the first row in the test dataset (1, 168, 7)
# target is the set of n=forecast_window features at the end of the test dataset
# that means the 24 values to predict, along with their positional encodings

In [75]:
prediction = model.predict(
    next_input_model, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False
)

In [76]:
# shape of prediction is prediction_batch_size, n_timesteps, value_feature_only (no positional encodings) 
prediction.shape

(1, 168, 1)

In [77]:
# get the value of the most recent prediction (last timestep) into the predictions list
predictions_list.append(prediction[:, -1, :][0][0])

In [78]:
len(predictions_list)

1

In [80]:
# from source tensor, get the positional encodings for ti+1 to t167 (that is 168-i-1 values)
pos_encoding_old_values = src[:, i+1:, 1:]
pos_encoding_old_values.shape

TensorShape([1, 167, 6])

In [90]:
# from target tensor, get the positional encodings for t168 to t168+i (that is i+1 values)
pos_encoding_new_val = target[:, :i+1, 1:]
pos_encoding_new_val.shape

TensorShape([1, 1, 6])

In [91]:
# build new positional encodings with 168 values
pos_encodings = tf.concat([pos_encoding_old_values, pos_encoding_new_val], axis=1)
pos_encodings = tf.cast(pos_encodings, dtype=tf.float32)
pos_encodings.shape

TensorShape([1, 168, 6])

In [92]:
# build the values feature for the next input to the model
# pop i+1 values at the beginning of the previous input
value_feature_old_values = tf.expand_dims(src[:, i+1:, 0], axis=-1)
value_feature_old_values = tf.cast(value_feature_old_values, dtype=tf.float32)
value_feature_old_values.shape

TensorShape([1, 167, 1])

In [103]:
# current predictions_list to NumPy array
value_feature_new_values = np.array(predictions_list[:i+1])
value_feature_new_values

array([0.33857653], dtype=float32)

In [104]:
# current prediction array to tensor
value_feature_new_values = tf.convert_to_tensor(value_feature_new_values)
value_feature_new_values

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.33857653], dtype=float32)>

In [105]:
# expand dimensions of current prediction tensor to single-value feature
value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=-1)
value_feature_new_values

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.33857653]], dtype=float32)>

In [106]:
# expand dimensions of current prediction tensor to single-value batch
value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=0)
value_feature_new_values

<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[0.33857653]]], dtype=float32)>

In [110]:
next_input_model = tf.concat([value_feature_old_values, value_feature_new_values], axis=1)
next_input_model.shape

TensorShape([1, 168, 1])

In [111]:
next_input_model = tf.concat([next_input_model, pos_encodings], axis=2)
next_input_model.shape

TensorShape([1, 168, 7])

In [112]:
# second pass
i = 1

In [113]:
prediction = model.predict(
    next_input_model, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False
)

In [114]:
# shape of prediction is prediction_batch_size, n_timesteps, value_feature_only (no positional encodings) 
prediction.shape

(1, 168, 1)

In [115]:
# get the value of the most recent prediction (last timestep) into the predictions list
predictions_list.append(prediction[:, -1, :][0][0])

In [116]:
len(predictions_list)

2

In [118]:
predictions_list

[0.33857653, 0.4081602]

In [119]:
# from source tensor, get the positional encodings for ti+1 to t167 (that is 168-i-1 values)
pos_encoding_old_values = src[:, i+1:, 1:]
pos_encoding_old_values.shape

TensorShape([1, 166, 6])

In [120]:
# from target tensor, get the positional encodings for t168 to t168+i (that is i+1 values)
pos_encoding_new_val = target[:, :i+1, 1:]
pos_encoding_new_val.shape

TensorShape([1, 2, 6])

In [121]:
# build new positional encodings with 168 values
pos_encodings = tf.concat([pos_encoding_old_values, pos_encoding_new_val], axis=1)
pos_encodings = tf.cast(pos_encodings, dtype=tf.float32)
pos_encodings.shape

TensorShape([1, 168, 6])

In [122]:
# build the values feature for the next input to the model
# pop i+1 values at the beginning of the previous input
value_feature_old_values = tf.expand_dims(src[:, i+1:, 0], axis=-1)
value_feature_old_values = tf.cast(value_feature_old_values, dtype=tf.float32)
value_feature_old_values.shape

TensorShape([1, 166, 1])

In [123]:
# current predictions_list to NumPy array
value_feature_new_values = np.array(predictions_list[:i+1])
value_feature_new_values

array([0.33857653, 0.4081602 ], dtype=float32)

In [124]:
value_feature_new_values = tf.convert_to_tensor(value_feature_new_values)
value_feature_new_values

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.33857653, 0.4081602 ], dtype=float32)>

In [125]:
# expand dimensions of current prediction tensor to single-value feature
value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=-1)
value_feature_new_values

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.33857653],
       [0.4081602 ]], dtype=float32)>

In [126]:
# expand dimensions of current prediction tensor to single-value batch
value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=0)
value_feature_new_values

<tf.Tensor: shape=(1, 2, 1), dtype=float32, numpy=
array([[[0.33857653],
        [0.4081602 ]]], dtype=float32)>

In [127]:
next_input_model = tf.concat([value_feature_old_values, value_feature_new_values], axis=1)
next_input_model.shape

TensorShape([1, 168, 1])

In [128]:
next_input_model = tf.concat([next_input_model, pos_encodings], axis=2)
next_input_model.shape

TensorShape([1, 168, 7])

In [129]:
# the iterative cycle has been successfully tested, build the final code in a new notebook